In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow.contrib.rnn as rnn
import numpy as np
import os

In [2]:
mnist = input_data.read_data_sets('./data/mnist',one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.train.images.shape

(55000, 784)

In [4]:
mnist.test.images.shape

(10000, 784)

In [5]:
mnist.train.labels.dtype

dtype('float64')

In [6]:
mnist.train.labels.shape

(55000, 10)

2.构建computation graph

In [7]:
#placeholder
#超参数
batch_size =128
n_input = 28
n_steps = 784//n_input#(mnist.train.images.shape[1])/n_input
class_num = 10


with tf.name_scope('data'):
    X = tf.placeholder(tf.float32,shape = [batch_size,n_input*n_steps],name = "X")
    Y = tf.placeholder(tf.uint8,shape = [batch_size,class_num])
    reshaped_X = tf.reshape(X,[batch_size,n_steps,n_input],name='reshaped_X')
    unstacked_X = tf.unstack(reshaped_X,n_steps,1,name='unstack_X')#list of tensor(batch,n_input)

In [8]:
#computation map
hidden_unit_size = 128
with tf.name_scope('lstm'):
    #weight and bias for the final output of the network
    weight = tf.Variable(tf.random_normal([hidden_unit_size,class_num]),name="weight_for_output")
    bias   = tf.Variable(tf.zeros([class_num]),name='bias_for_output')
    
    lstm_cell = rnn.BasicLSTMCell(hidden_unit_size,forget_bias=1.0)
    outputs,state = rnn.static_rnn(lstm_cell,unstacked_X,dtype=tf.float32)
    pred = tf.add(tf.matmul(outputs[-1],weight),bias,name='pred')

In [9]:
#loss function
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=pred),name="loss")
    
    

In [10]:
#optimizer
learning_rate = 0.001
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [11]:
#evaluate
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.arg_max(pred,1),tf.arg_max(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [12]:
#start training
epoch = 10000
display_step = 1000
save_step = 1000

saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter('./graphs/LSTM_MNIST',sess.graph)
    
    
    for i in range(epoch):
        batch_X,batch_Y = mnist.train.next_batch(batch_size=batch_size)
        sess.run([optimizer],feed_dict={X:batch_X,Y:batch_Y})
        if i % display_step == 0:
            # 计算batch上的准确率
            acc = sess.run(accuracy, feed_dict={X: batch_X, Y: batch_Y})
            # 计算batch上的loss
            l = sess.run(loss, feed_dict={X: batch_X, Y: batch_Y})
            print("Iter " + str(i) + ", Minibatch Loss= " + \
                  "{:.6f}".format(l) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            #保存训练结果
            saver.save(sess,'./checkpoints/LSTM/LSTM-MNIST',global_step=i)
            print("saved at iteration:{}".format(i))
    print("Optimization Finished!")
    writer.close() 

Iter 0, Minibatch Loss= 2.156565, Training Accuracy= 0.24219
saved at iteration:0
Iter 1000, Minibatch Loss= 0.071404, Training Accuracy= 0.96875
saved at iteration:1000
Iter 2000, Minibatch Loss= 0.056611, Training Accuracy= 0.98438
saved at iteration:2000
Iter 3000, Minibatch Loss= 0.023989, Training Accuracy= 1.00000
saved at iteration:3000
Iter 4000, Minibatch Loss= 0.027956, Training Accuracy= 0.99219
saved at iteration:4000
Iter 5000, Minibatch Loss= 0.016840, Training Accuracy= 0.99219
saved at iteration:5000
Iter 6000, Minibatch Loss= 0.001119, Training Accuracy= 1.00000
saved at iteration:6000
Iter 7000, Minibatch Loss= 0.014666, Training Accuracy= 0.99219
saved at iteration:7000
Iter 8000, Minibatch Loss= 0.002093, Training Accuracy= 1.00000
saved at iteration:8000
Iter 9000, Minibatch Loss= 0.003177, Training Accuracy= 1.00000
saved at iteration:9000
Optimization Finished!


In [13]:
#evaluate the model on test set
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/LSTM/LSTM-MNIST'))
    if ckpt is not None:
        checkpoint_path = ckpt.model_checkpoint_path
        saver.restore(sess,checkpoint_path)
    batch_X,batch_Y = mnist.train.next_batch(batch_size)
    feed_dict = {X:batch_X,Y:batch_Y}
    l,acc = sess.run([loss,accuracy],feed_dict=feed_dict)
    print("accuracy for test set batch:{} ".format(acc))

INFO:tensorflow:Restoring parameters from ./checkpoints/LSTM/LSTM-MNIST-9000
accuracy for test set batch:1.0 
